In [36]:
import datetime
import numpy as np
import pandas as pd
import yfinance as yf
import warnings
from arch import arch_model
from pmdarima.arima import auto_arima
warnings.filterwarnings('ignore')

In [37]:
def day_price_ret(ticker_symbol,s_date,e_date):
    stock_symbol = ticker_symbol

    stock_data = yf.download(stock_symbol, start=s_date, end=e_date)
    stock_data = stock_data.set_index(pd.to_datetime(stock_data.index))

    weekly_returns = stock_data['Adj Close'].pct_change().dropna()

    return pd.DataFrame(weekly_returns)

In [47]:
s_date = datetime.datetime(2022,4,1)
e_date = datetime.datetime(2024,4,1)
kos = day_price_ret('^KS11',s_date,e_date)['Adj Close']
df = day_price_ret('395400.ks',s_date,e_date)['Adj Close']

[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed


In [49]:
model = arch_model(df, vol='GARCH', p=1, q=1)
result = model.fit(disp='off')


forecast_volatility = result.forecast(horizon=1)

last_volatility = forecast_volatility.mean.iloc[-1]

correlation = np.corrcoef(df, kos)[0, 1]

beta = correlation * (last_volatility / kos.std())

print("Beta:", beta)

Beta: h.1   -0.006921
Name: 2024-03-29 00:00:00, dtype: float64


In [50]:
## 금리 인상기에 비정상상성을 띄는 종목들이 있는데 그 종목들만 정상적으로 변경해서 garch에 넣어야 하는지 비교군들 또한 같은 전처리 과정이 필요한지?
## garch 값의 최적의 p와 q를 고르는 법
## garch 에서 나온 변동성을 통해 beta값을 구할때 kospi의 표준편차로 나워야하는지? 리츠의 표준편차로 나눠야 하는지?
## garch 모형이 적합한지 적합성을 검정하는 법